<a href="https://colab.research.google.com/github/practice-dump/practice-dump/blob/main/Trainer_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In this notebook we would explore and easy and sophisticated way to train transformer using **Trainer ** library

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="save_folder/",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
)

Training Arguments contains all hyperparmeters which can be changed as per wish/hyperparmeter tuning exercise. If no value is given than default value is taken

In [ ]:
#training_args

One can all training_args by uncommenting above command

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Loading the tokenizer

In [7]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["text"]) ##Here one can add other relevant arguments

Function to tokenize ** text** column in dataset

We will use map function to tokenize all dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rotten_tomatoes")  # doctest: +IGNORE_RESULT

Downloading dataset

In [13]:
dataset['train']['text'][0],dataset['train']['label'][0]

('the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 1)

In [14]:
dataset = dataset.map(tokenize_dataset, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Here we have tokenized all dataset

In [15]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) ##This is something new for me would explore this later 
##I guess the script which I use contains an argument "batch_size" when running map command so maybe that does the same purpose

##Would update this later when I get more information

Used to create a batch of examples from your dataset

In [ ]:
model.cuda()

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)  # doctest: +SKIP

So here we have told Trainer class the model, training_arg,train_dataset,eval_dataset, which tokenizer we are using,data_collator

Here you can also set loss function, optimizer, scheduler

In [18]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8530
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2134
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster tha

Saving model checkpoint to save_folder/checkpoint-500
Configuration saved in save_folder/checkpoint-500/config.json
Model weights saved in save_folder/checkpoint-500/pytorch_model.bin
tokenizer config file saved in save_folder/checkpoint-500/tokenizer_config.json
Special tokens file saved in save_folder/checkpoint-500/special_tokens_map.json
Saving model checkpoint to save_folder/checkpoint-1000
Configuration saved in save_folder/checkpoint-1000/config.json
Model weights saved in save_folder/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in save_folder/checkpoint-1000/tokenizer_config.json
Special tokens file saved in save_folder/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to save_folder/checkpoint-1500
Configuration saved in save_folder/checkpoint-1500/config.json
Model weights saved in save_folder/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in save_folder/checkpoint-1500/tokenizer_config.json
Special tokens file saved in save_folde

TrainOutput(global_step=2134, training_loss=0.3434138901343162, metrics={'train_runtime': 151.9632, 'train_samples_per_second': 112.264, 'train_steps_per_second': 14.043, 'total_flos': 196196118777384.0, 'train_loss': 0.3434138901343162, 'epoch': 2.0})

By running this command the actual training happens, and it shows results and saves checkpoints according to hyperparameters set in training_args

For tasks - like translation or summarization - that use a sequence-to-sequence model, use the Seq2SeqTrainer and Seq2SeqTrainingArguments classes instead.